# Writing data

In this example we write to a sqlite3 b using SQL Alchemy

See also:
* [SQL Alchemy Tutorial](https://docs.sqlalchemy.org/en/13/orm/tutorial.html)
* [sqlite at SQl Alchemy](https://docs.sqlalchemy.org/en/13/dialects/sqlite.html)


In [ ]:
import pandas as pd

import sqlite3

In [ ]:
import sqlalchemy
sqlalchemy.__version__

## Create the engine --> echo=True utiizes the integration with Python logging to display SQL
To make the output more brief set ```echo=False```.

In [ ]:
from sqlalchemy import create_engine
# conn_string = 'sqlite:///etl.db'
conn_string = 'sqlite:///:memory:'
engine = create_engine(conn_string, echo=True)

## Declare a Base from which concrete classes should extend.

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## Define a class that provides the OR/M - mapping

In [ ]:
from sqlalchemy import Column, Date, Float, Integer, String
class Weather(Base):
    __tablename__ = 'weather'

    id = Column(Integer, primary_key=True)

    city = Column(String)
    date = Column(Date)
    actual_mean_temp = Column(Integer)
    actual_min_temp = Column(Integer)
    actual_max_temp = Column(Integer)

    actual_precipitation = Column(Float)
    average_precipitation = Column(Float)
    record_precipitation = Column(Float)

    reserved2 = Column(String)

    # this is optional - only here for easily reading output later in this notebook
    def __repr__(self):
        return "<Weather(id=%d, date='%s', actual_mean_temp=%d, actual_min_temp=%d, actual_max_temp=%d, actual_precipitation=%f, average_precipitation=%f, record_precipitation=%f, reserved2='%s')>" % (
            self.id,
            self.date,
            self.actual_mean_temp,
            self.actual_min_temp,
            self.actual_max_temp,
            self.actual_precipitation,
            self.average_precipitation,
            self.record_precipitation,
            self.reserved2
            )

In [ ]:
Weather.__table__

In [ ]:
Base.metadata.create_all(engine)

## The resulting class is a POPO

In [ ]:
from datetime import datetime
w1 = Weather(
    id=1,
    date=datetime.strptime('2020-05-24', '%Y-%m-%d'),
    actual_mean_temp=82, actual_min_temp=57, actual_max_temp=84,
    actual_precipitation=0.0, average_precipitation=0.0, record_precipitation=0.0,
    reserved2='reserved2')
w1

In [ ]:
w1.date

In [ ]:
w1.actual_max_temp

In [ ]:
w1.id

## Make a session

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

## Add record

In [ ]:
session.add(w1)

## Cause flush by querying

In [ ]:
our_weather = session.query(Weather).filter_by(id=1).first()
our_weather

## Clear db

In [ ]:
session.query(Weather).delete()
session.query(Weather).count()

## Populate from file

_Note that we can select fields that we want and ignore the rest. This will make it easy to absorb file format changes._


In [ ]:
df_with_reserved = pd.read_table('../data/KPHX_with_reserved.dat', sep='|', header=0,
    usecols=[
        'id','city','date','actual_mean_temp','actual_min_temp','actual_max_temp','actual_precipitation','average_precipitation','record_precipitation','reserved2'
        ],
    dtype = {
        'id': 'int', 'city': 'string',
        #'date': '?', # let parser handle the converion - see https://stackoverflow.com/questions/21269399/datetime-dtypes-in-pandas-read-csv
        'actual_mean_temp': 'int8', 'actual_min_temp': 'int8','actual_max_temp': 'int8',
        # 'actual_precipitation','average_precipitation','record_precipitation',
        'reserved2': 'string'
    },
    # These next three are needed to parse and optimize datetime input handling
    parse_dates = [2],
    infer_datetime_format = True,
    date_parser = pd.to_datetime
)

for index, row in df_with_reserved.iterrows():
    w = Weather(**row)
    session.add(w)

session.commit()

### Should see 365 rows

In [ ]:
print('Should have 365 rows.')
session.query(Weather).count()

### All the rows

In [ ]:
for row in session.query(Weather):
    print(row)